# Working with geomedians

Geomedians are 

*see https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/master/Frequently_used_code/Generating_geomedian_composites.ipynb for useful material on defining geomedians. Probably need to simplify and condense the information. Aim: users understand why to use geomedian and how to use it*

*Would have been mentioned in the Time video from the previous section*

* scientifically rigorous
* better for spectral indices, which we will see next session

## `xarray` functions mentioned in trello

There is a trello card for Week 3: Explain xarray timeseries

Not sure where that best fits in, possibly in this section.

*Proposal: function explanations are best paired with immediate exercise/example. These two xarray items might work well with the Week 4 content on spectral indices. Consider moving to Week 4?*

## Optional activity

*Reference composites notebook to see other examples of composites (possibly [this notebook](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/master/Frequently_used_code/Generating_composites.ipynb) or [this notebook](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/master/Frequently_used_code/Generating_geomedian_composites.ipynb), I have not investigated thoroughly)*